In [1]:
from PolConfML.search import Search_TiO2 as Search, search_extended_config_TiO2 as search_extended_config
from PolConfML.model import Network, Dataset
from PolConfML.utils import load_descs, save_descs
import torch
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
path = 'data_TiO2/split/'

keys = ['S0_A','S0_B','S1_A','S1_B','S2_A','S2_B']

# train
descs_train, indices_train = load_descs(path+'train/',keys)
Y_train = np.array(np.load(path+'train/Y.npy'))
defects_train = np.load(path+'train/defect.npy')
n = [descs_train[site].shape[0] for site in descs_train]

# data
data_train = Dataset(descs_train, indices_train, Y_train, defects_train)
data_train.kernel(0.3)

In [3]:
net = torch.load('data_TiO2/model.pt')

In [4]:
box_size = np.diag(np.loadtxt('data_TiO2/POSCAR',skiprows=2,max_rows=3))

ov_pos = np.array([
        [0.666666666667, 0.75, 0.5242],
        [0.000000000000, 0.25, 0.5242],
        [0.444444444444, 0.25, 0.5242],
        [0.222222222222, 0.75, 0.5242],
        [0.000000000000, 0.75, 0.5242],
        [0.777777777778, 0.25, 0.5242],
        [0.444444444444, 0.75, 0.5242],
        [0.222222222222, 0.25, 0.5242],
        [0.555555555556, 0.25, 0.5242]])*box_size

In [5]:
s = Search('data_TiO2/POSCAR')

In [6]:
import time
start = time.time()
site_list = ['S0_A','S0_B', 'S1_A', 'S1_B', 'S2_A','S2_B']
res = {}
a, b = search_extended_config(net, data_train, s, np.array([]),2, ov_pos[:1], site_list, box_size)
res[1]= {'idxs':a, 'en':b}
# controls number of configurations for constructing more defective configurations
# 1 for a very rough search; 100 for better results
n_search = 1

for j in range(2,10):
    print(j,'\t')
    tmp_idxs = []
    tmp_en = []
    for i in range(n_search):
        print(i/n_search,'\t',end='\r')
        a, b = search_extended_config(net, data_train, s, res[j-1]['idxs'][i], 2, ov_pos[:j], site_list, box_size)
        tmp_idxs.append(a)
        tmp_en.append(b)
    tmp_idxs = np.concatenate(tmp_idxs)
    tmp_idxs = np.sort(tmp_idxs, axis=1)
    tmp_en = np.concatenate(tmp_en)
    _,idxs = np.unique(tmp_idxs,axis=0,return_index=True)
    print(len(idxs),'/',len(tmp_en))
    tmp_idxs = tmp_idxs[idxs]
    tmp_en = tmp_en[idxs]
    idxs = np.argsort(tmp_en)
    res[j] = {'idxs':tmp_idxs[idxs], 'en':tmp_en[idxs]}
end = time.time()
print(end-start)

2 	
5565 / 5565
3 	
5356 / 5356
4 	
5151 / 5151
5 	
4950 / 4950
6 	
4753 / 4753
7 	
4560 / 4560
8 	
4371 / 4371
9 	
4186 / 4186
282.8688690662384


In [7]:
for i in res:
    print(i, 'VO')
    for idx in res[i]['idxs'][:3]:
        print(idx)

1 VO
[ 58 158]
[ 68 148]
[ 78 148]
2 VO
[ 58  88  98 158]
[ 58  88 108 158]
[ 58  88 158 178]
3 VO
[ 28  58  88  98 144 158]
[ 38  58  88  98 144 158]
[ 28  58  88  98 128 158]
4 VO
[ 28  34  58  88  98 104 144 158]
[ 28  58  88  98 104 128 144 158]
[ 14  28  58  88  98 128 144 158]
5 VO
[ 28  34  58  84  88  98 104 128 144 158]
[ 14  28  34  58  88  98 104 128 144 158]
[ 28  34  43  58  84  88  98 104 144 158]
6 VO
[ 28  34  58  84  88  89  98 104 113 128 144 158]
[ 28  34  58  84  88  89  93  98 104 128 144 158]
[ 28  34  43  58  84  88  98 104 128 144 158 174]
7 VO
[ 28  34  54  58  84  88  89  98 104 113 128 144 158 174]
[ 28  34  58  84  88  89  98 104 113 128 133 144 158 174]
[ 14  28  34  58  84  88  89  98 104 113 128 144 158 174]
8 VO
[ 14  28  34  54  58  73  84  88  89  98 104 113 128 144 158 174]
[ 14  28  34  54  58  84  88  89  98 104 113 128 133 144 158 174]
[ 14  28  34  54  58  84  88  89  98 104 113 128 144 158 163 174]
9 VO
[  9  14  28  34  54  58  73  84  88  89  9